# Projectile Motion - Advanced Features

This notebook demonstrates advanced Funz features:
- 2D parameter space exploration
- Result caching for efficient re-runs
- Contour plots and heatmaps
- Finding optimal parameters

## Problem: Hit a Target

Given a target at distance $d$ and height $h$, what combination of velocity and angle will hit it?

We'll explore the 2D parameter space (velocity × angle) to find all valid combinations.

## 1. Create the Modelica Model

In [ ]:
# Create the parametric Modelica model
model_content = '''model ProjectileMotion
  "Parametric projectile motion model"
  
  // Parametric variables (can be varied by Funz)
  parameter Real v0 = ${velocity~20.0} "Initial velocity (m/s)";
  parameter Real angle = ${launch_angle~45.0} "Launch angle (degrees)";
  
  // Fixed parameters
  parameter Real g = 9.81 "Gravitational acceleration (m/s^2)";
  parameter Real m = 1.0 "Mass (kg)";
  
  // State variables
  Real x(start = 0.0) "Horizontal position (m)";
  Real y(start = 0.0) "Vertical position (m)";
  Real vx(start = v0 * cos(angle * 3.14159265359 / 180.0)) "Horizontal velocity (m/s)";
  Real vy(start = v0 * sin(angle * 3.14159265359 / 180.0)) "Vertical velocity (m/s)";
  
equation
  // Equations of motion
  der(x) = vx;
  der(y) = vy;
  der(vx) = 0;  // No horizontal acceleration
  der(vy) = -g; // Gravitational acceleration downward
  
end ProjectileMotion;
'''

with open('ProjectileMotion.mo', 'w') as f:
    f.write(model_content)

print("✓ Model created with parametric variables: velocity, launch_angle")

## 2. Explore 2D Parameter Space

Let's run a comprehensive parameter sweep across velocity and angle.

In [ ]:
import fz
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Define parameter grid
velocities = np.linspace(10, 30, 9)  # 9 velocities from 10 to 30 m/s
angles = np.linspace(15, 75, 9)      # 9 angles from 15° to 75°

print(f"Running 2D parameter sweep...")
print(f"  Velocities: {len(velocities)} values from {velocities[0]:.1f} to {velocities[-1]:.1f} m/s")
print(f"  Angles: {len(angles)} values from {angles[0]:.1f}° to {angles[-1]:.1f}°")
print(f"  Total simulations: {len(velocities) * len(angles)}")

# Run parameter sweep
results_2d = fz.fzr(
    "ProjectileMotion.mo",
    {"velocity": velocities.tolist(), 
     "launch_angle": angles.tolist()},
    "Modelica",
    calculators="localhost",
    results_dir="results_2d_sweep"
)

print(f"\n✓ Completed {len(results_2d)} simulations")
print(f"  Success rate: {sum(results_2d['status'] == 'done')} / {len(results_2d)}")

display(results_2d[['velocity', 'launch_angle', 'status']].head(10))

## 3. Extract Range and Height Data

Calculate the range and maximum height for each parameter combination.

In [ ]:
# Extract metrics from each simulation
sweep_data = []

for idx, row in results_2d.iterrows():
    if row['status'] == 'done' and 'res' in row:
        velocity = row['velocity']
        angle = row['launch_angle']
        res_data = row['res']['ProjectileMotion']
        
        x_vals = np.array(list(res_data['x'].values()))
        y_vals = np.array(list(res_data['y'].values()))
        
        # Find landing point (where y crosses 0)
        landing_idx = np.where(y_vals < 0)[0]
        if len(landing_idx) > 0:
            landing_idx = landing_idx[0] - 1
        else:
            landing_idx = len(y_vals) - 1
        
        range_distance = x_vals[landing_idx]
        max_height = np.max(y_vals[y_vals >= 0])
        
        sweep_data.append({
            'velocity': velocity,
            'angle': angle,
            'range': range_distance,
            'max_height': max_height
        })

sweep_df = pd.DataFrame(sweep_data)
print(f"\nExtracted data for {len(sweep_df)} successful simulations")
display(sweep_df.head(10))

## 4. Create Contour Plots

Visualize how range and height vary across the 2D parameter space.

In [ ]:
from scipy.interpolate import griddata

# Create meshgrid for contour plots
v_grid = np.linspace(sweep_df['velocity'].min(), sweep_df['velocity'].max(), 100)
a_grid = np.linspace(sweep_df['angle'].min(), sweep_df['angle'].max(), 100)
V, A = np.meshgrid(v_grid, a_grid)

# Interpolate range and height data
points = sweep_df[['velocity', 'angle']].values
range_grid = griddata(points, sweep_df['range'].values, (V, A), method='cubic')
height_grid = griddata(points, sweep_df['max_height'].values, (V, A), method='cubic')

# Create figure
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Range contour plot
contour1 = ax1.contourf(V, A, range_grid, levels=20, cmap='viridis')
ax1.contour(V, A, range_grid, levels=10, colors='white', alpha=0.3, linewidths=0.5)
ax1.scatter(sweep_df['velocity'], sweep_df['angle'], c='red', s=30, 
           edgecolors='white', linewidths=1, alpha=0.6, label='Simulation points')
cbar1 = plt.colorbar(contour1, ax=ax1)
cbar1.set_label('Range (m)', fontsize=11)
ax1.set_xlabel('Initial Velocity (m/s)', fontsize=12)
ax1.set_ylabel('Launch Angle (degrees)', fontsize=12)
ax1.set_title('Range as Function of Velocity and Angle', fontsize=13, fontweight='bold')
ax1.legend(loc='upper left')
ax1.grid(True, alpha=0.2)

# Maximum height contour plot
contour2 = ax2.contourf(V, A, height_grid, levels=20, cmap='plasma')
ax2.contour(V, A, height_grid, levels=10, colors='white', alpha=0.3, linewidths=0.5)
ax2.scatter(sweep_df['velocity'], sweep_df['angle'], c='red', s=30,
           edgecolors='white', linewidths=1, alpha=0.6, label='Simulation points')
cbar2 = plt.colorbar(contour2, ax=ax2)
cbar2.set_label('Max Height (m)', fontsize=11)
ax2.set_xlabel('Initial Velocity (m/s)', fontsize=12)
ax2.set_ylabel('Launch Angle (degrees)', fontsize=12)
ax2.set_title('Maximum Height as Function of Velocity and Angle', fontsize=13, fontweight='bold')
ax2.legend(loc='upper left')
ax2.grid(True, alpha=0.2)

plt.tight_layout()
plt.savefig('parameter_space_contours.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Contour plots saved as 'parameter_space_contours.png'")

## 5. Hit a Target - Find Valid Parameters

Let's say we want to hit a target at distance = 40m and height = 5m.
Which combinations of velocity and angle will work?

In [ ]:
# Define target
target_distance = 40  # meters
target_height = 5     # meters
tolerance = 2         # meters (acceptable error)

print(f"Target: distance = {target_distance}m, height = {target_height}m")
print(f"Tolerance: ±{tolerance}m\n")

# Find parameter combinations that hit the target
# We need to check if the trajectory passes through the target zone
# For simplicity, we'll check if:
#  - range ≈ target_distance
#  - max_height >= target_height

valid_params = sweep_df[
    (sweep_df['range'] >= target_distance - tolerance) &
    (sweep_df['range'] <= target_distance + tolerance) &
    (sweep_df['max_height'] >= target_height)
].copy()

print(f"Found {len(valid_params)} parameter combinations that can hit the target:\n")
display(valid_params.round(2))

if len(valid_params) > 0:
    print(f"\n🎯 Recommended solutions:")
    
    # Find solution with minimum velocity (most efficient)
    min_v_idx = valid_params['velocity'].idxmin()
    print(f"  • Minimum velocity: v₀ = {valid_params.loc[min_v_idx, 'velocity']:.1f} m/s, "
          f"θ = {valid_params.loc[min_v_idx, 'angle']:.1f}°")
    
    # Find solution closest to 45° (most robust)
    valid_params['angle_diff'] = abs(valid_params['angle'] - 45)
    mid_angle_idx = valid_params['angle_diff'].idxmin()
    print(f"  • Closest to 45°: v₀ = {valid_params.loc[mid_angle_idx, 'velocity']:.1f} m/s, "
          f"θ = {valid_params.loc[mid_angle_idx, 'angle']:.1f}°")
else:
    print("\n❌ No valid parameter combinations found with current grid.")
    print("   Try: 1) Finer grid, 2) Different velocity/angle ranges, 3) Larger tolerance")

## 6. Visualize Target Hitting Zone

In [ ]:
fig, ax = plt.subplots(figsize=(14, 8))

# Plot range contours
contour = ax.contourf(V, A, range_grid, levels=20, cmap='coolwarm', alpha=0.6)
contour_lines = ax.contour(V, A, range_grid, 
                           levels=[target_distance - tolerance, target_distance, target_distance + tolerance],
                           colors=['blue', 'black', 'blue'], linewidths=[2, 3, 2],
                           linestyles=['--', '-', '--'])
ax.clabel(contour_lines, inline=True, fontsize=10, fmt='%0.1f m')

# Mark all simulation points
ax.scatter(sweep_df['velocity'], sweep_df['angle'], c='gray', s=40,
          edgecolors='white', linewidths=1, alpha=0.4, label='All simulations')

# Highlight valid parameters
if len(valid_params) > 0:
    ax.scatter(valid_params['velocity'], valid_params['angle'], c='lime', s=150,
              edgecolors='darkgreen', linewidths=2, marker='*', 
              label=f'Valid solutions (n={len(valid_params)})', zorder=10)

# Mark target zone
ax.axhline(y=45, color='red', linestyle=':', linewidth=2, alpha=0.5, label='45° optimal')

cbar = plt.colorbar(contour, ax=ax)
cbar.set_label('Range (m)', fontsize=12)

ax.set_xlabel('Initial Velocity (m/s)', fontsize=13)
ax.set_ylabel('Launch Angle (degrees)', fontsize=13)
ax.set_title(f'Hit Target at {target_distance}m distance, {target_height}m height (±{tolerance}m tolerance)',
            fontsize=14, fontweight='bold')
ax.legend(fontsize=11, loc='upper left')
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('target_hitting_zone.png', dpi=150, bbox_inches='tight')
plt.show()

print("\n✓ Target analysis saved as 'target_hitting_zone.png'")

## 7. Using Cache for Efficient Re-runs

Now let's demonstrate caching. If we want to add more parameter values, Funz will reuse previous results.

In [ ]:
# Add more points to the grid
velocities_refined = np.linspace(10, 30, 13)  # More points
angles_refined = np.linspace(15, 75, 13)      # More points

print(f"Re-running with refined grid...")
print(f"  Previous: {len(velocities)}×{len(angles)} = {len(velocities)*len(angles)} simulations")
print(f"  New: {len(velocities_refined)}×{len(angles_refined)} = {len(velocities_refined)*len(angles_refined)} simulations")
print(f"  Expected from cache: ~{len(velocities)*len(angles)} results")
print(f"  Expected new: ~{len(velocities_refined)*len(angles_refined) - len(velocities)*len(angles)} simulations\n")

# Use cache from previous run
results_refined = fz.fzr(
    "ProjectileMotion.mo",
    {"velocity": velocities_refined.tolist(),
     "launch_angle": angles_refined.tolist()},
    "Modelica",
    calculators=["cache://results_2d_sweep", "localhost"],  # Use cache first!
    results_dir="results_2d_refined"
)

# Count cache hits
cache_hits = sum(results_refined['calculator'].str.contains('cache://', na=False))
new_calcs = len(results_refined) - cache_hits

print(f"\n✓ Results:")
print(f"  From cache: {cache_hits} simulations (saved time!)")
print(f"  New calculations: {new_calcs} simulations")
print(f"  Total: {len(results_refined)} results")
print(f"\n💾 Cache efficiency: {cache_hits/len(results_refined)*100:.1f}% of results reused")

display(results_refined[['velocity', 'launch_angle', 'status', 'calculator']].head(15))

## 8. Summary

In this notebook, we demonstrated:

### Advanced Funz Features:
1. **2D Parameter Space Exploration**: Systematic sweep of velocity × angle
2. **Contour Visualization**: Understanding parameter effects with heatmaps
3. **Target Optimization**: Finding parameter combinations to meet objectives
4. **Result Caching**: Efficient re-runs by reusing previous calculations

### Key Insights:
- **Range peaks at 45°** for all velocities (classical mechanics validation)
- **Multiple solutions** exist for hitting a target (high/low trajectory tradeoff)
- **Cache efficiency** dramatically reduces computation time for iterative studies

### Practical Applications:
- Ballistics and sports (golf, basketball, artillery)
- Robot arm trajectory planning
- Package delivery drones
- Water fountain design

## Next Steps

Extend this analysis by:
- Adding air resistance (quadratic drag)
- Optimization algorithms (gradient descent, genetic algorithms)
- Moving targets
- Wind effects
- Multi-objective optimization (minimize energy AND hit target)